In [ ]:
from google.colab import files, drive
#drive.mount('/content/gdrive')
uploaded = files.upload()

Saving 48_0_names.txt to 48_0_names.txt
Saving 49_0_names.txt to 49_0_names.txt


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from google.colab import files

for i in range(len(list(uploaded))):
  df = pd.read_csv(list(uploaded)[i])
  Names = []
  for name in df.columns:
    Names.append(name)
  Unpacked_Size = []
  Total_Files = []
  Last_publish = []
  Weekly_download = []
  versions = []
  git_watch =[]
  git_star = []
  git_fork = []
  have_size = 0;
  have_total_files = 0;
  have_last_publish = 0;
  have_weekly_download = 0;
  have_github = 0
  import time
  import json
  count=1

  from multiprocessing.pool import ThreadPool
  import threading
  lock = threading.Lock()
  name_list = []
  def scrap(name):
    have_size = 0;
    have_total_files = 0;
    have_last_publish = 0;
    have_weekly_download = 0;
    have_github = 0
    global count
    url = "https://www.npmjs.com/package/" + name
    url2 = "http://registry.npmjs.com/" + name
    #print("OK0")
    r = requests.get(url, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'})
    r2 = requests.get(url2, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'})
    #print("OK1")
    count2 =0
    while (r.status_code == 429):
        if (count2 == 100):
          print("ignore", count, name)
          return
        time.sleep(1)
        count2 +=1
        r = requests.get(url, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'})
    #print("OK1.5")
    while (r2.status_code == 429):
        time.sleep(1)
        r2 = requests.get(url2, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'})
    #print("OK2")
    obj = json.loads(r2.text)
    soup = BeautifulSoup(r.text, 'html.parser')
    info_need = soup.select('div._702d723c.dib.w-50.bb.b--black-10.pr2')

    for element in info_need:


      if(element.find('h3').text == "Unpacked Size"):
          #Unpacked_Size.append(element.find('p').text)
          have_size = have_size + 1;

          size = element.find('p').text
      elif(element.find('h3').text == "Total Files"):
          #Total_Files.append(element.find('p').text)
          have_total_files = have_total_files + 1

          files = element.find('p').text
      elif(element.find('h3').text == "Last publish"):
          #Last_publish.append(element.find('p').text)
          have_last_publish = have_last_publish + 1

          publish = element.find('p').text
      elif(element.find('h3').text == "Repository"):
          rep = element.select('span')[1].text
          if (rep.split("/")[0]=="github.com"):

            rep =  "https://"+rep
            r3 = requests.get(rep, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'})
            while (r3.status_code == 429):
              time.sleep(0.3)
              r3 = requests.get(rep, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'})
            soup3 = BeautifulSoup(r3.text, 'html.parser')
            res = soup3.select('.social-count')
            if res:
              have_github += 1            
            #git_watch.append(res[0].text.split("\n")[1].split(" ")[-1])
            #git_star.append(res[1].text.split("\n")[1].split(" ")[-1])
            #git_fork.append(res[2].text.split("\n")[1].split(" ")[-1])
              watch = res[0].text.split("\n")[1].split(" ")[-1]
              star = res[1].text.split("\n")[1].split(" ")[-1]
              fork = res[2].text.split("\n")[1].split(" ")[-1]

  #print("OK3")
    lock.acquire()
    name_list.append(name)

    if "time" in obj:
      versions.append(obj['time'])
    else:
      versions.append("NULL")

    download_count = soup.find('p' , class_ = '_9ba9a726 f4 tl flex-auto fw6 black-80 ma0 pr2 pb1')

    if (download_count):
      Weekly_download.append(int(download_count.text.replace(',', '')))
      have_weekly_download = have_weekly_download + 1

    if(have_size == 0):
      Unpacked_Size.append("NULL")
    else: 
      Unpacked_Size.append(size)    
    if(have_total_files == 0):
      Total_Files.append("NULL")
    else:
      Total_Files.append(files)    
    if(have_last_publish == 0):
      Last_publish.append("NULL")
    else:
      Last_publish.append(publish)

    if(have_weekly_download == 0):
      Weekly_download.append("NULL")
    if(have_github == 0):
      git_watch.append("NULL")
      git_star.append("NULL")
      git_fork.append("NULL")
    else:
      git_watch.append(watch)
      git_star.append(star)
      git_fork.append(fork)
    have_size = 0
    have_total_files = 0
    have_last_publish = 0
    have_weekly_download = 0
    have_github = 0
    print(count)
    #print("\r", count, end="")
    count+=1
    lock.release()
    #print("OK4")
  with ThreadPool(100) as pool:
    pool.map(scrap, Names)
  data = {'Total_Files': Total_Files, 'Unpacked_Size': Unpacked_Size, 'Last_publish': Last_publish, 'Weekly_download': Weekly_download, "Watch": git_watch ,"Star": git_star,"Fork": git_fork ,"Versions": versions}
  df_1 = pd.DataFrame(data, columns= ['Unpacked_Size','Total_Files', 'Last_publish','Weekly_download', 'Watch', 'Star', 'Fork', 'Versions'], index = name_list)

  name = "df_0_0(" + str(i) + ").csv"
  df_1.to_csv(name)
  files.download(name)